In [131]:
from pathlib import Path
from src.oxrivers_api.loader import Loader
from src.oxrivers_api.client import OxfordRiversClient
from src.oxrivers_api.sites_discovery import SitesDiscovery
from src.oxrivers_api.determinands_discovery import DeterminandsDiscovery

# choose where to store json
data_dir: Path = Path("../data")

# set up Oxford Rivers Client and Pandas loader
client = OxfordRiversClient(data_dir)
loader = Loader(client)

In [132]:
# Determinands discovery helps you figure out what datasets have the information you are looking for
determinands_discovery = DeterminandsDiscovery(loader)

# see what determinands there are
determinands_discovery.get_determinands_info()

# Escherichia coli (EC) is in the list. Let's see what datasets track E. coli.
ecoli_id = "Escherichia coli (EC)"
datasets_with_ecoli = determinands_discovery.get_datasets_with_determinand(ecoli_id)

Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/determinands
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/determinands
                                               name  \
0              Alkalinity to pH 4.5 as CaCO3 (mg/l)   
1                    Ammonia un-ionised as N (mg/l)   
2                                   Ammonium (mg/L)   
3                   Ammoniacal Nitrogen as N (mg/l)   
4                            Blue-Green Algae (RFU)   
5   Carbon, Organic, Dissolved as C :- {DOC} (mg/l)   
6                                   Chloride (mg/l)   
7                                Chlorophyll (µg/L)   
8                      Conductivity at 25 C (µS/cm)   
9               

In [133]:
# Sites discovery helps you see where the data you are interested in is collected
# for a particular dataset
sites_discoveries = [SitesDiscovery(loader, datasetID) for datasetID in datasets_with_ecoli["dataset"]]

# Let's look at the swimming areas
ea_bathing_sites_with_ecoli = sites_discoveries[0].get_sites_info()
ea_bathing_sites_columns = sites_discoveries[0].get_columns()
wolvercote_bathing_coordinates = sites_discoveries[0].sites["geometry_coordinates"].loc[0]


Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/sites
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/sites
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/sites
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/sites
  properties_id         properties_name
0         11946  Wolvercote Mill Stream
1         11947       Wallingford Beach
Index(['geometry_coordinates', 'properti

In [134]:
# Let's find other sites near Wolvercote in the other E. coli dataset
other_ecoli_sites_near_wolvercote = sites_discoveries[1].get_closest_sites_to(wolvercote_bathing_coordinates)
# And print the names of the ones within 10 km
within_10km_wolvercote = other_ecoli_sites_near_wolvercote[other_ecoli_sites_near_wolvercote["distance_from_ref"] < 10000]
print(within_10km_wolvercote["properties_name"])


Index(['geometry_coordinates', 'properties_id', 'properties_name',
       'properties_threshold', 'properties_popserved', 'lon', 'lat'],
      dtype='object')
['geometry_coordinates', 'properties_id', 'properties_name', 'properties_threshold', 'properties_popserved', 'lon', 'lat']
15    Tumbling Bay Bathing Area, Oxford (Laboratory ...
18                     Folly Bridge, Oxford (Petrifilm)
16                      Longbridges, Oxford (Petrifilm)
Name: properties_name, dtype: object


In [148]:
# Now let's plot the time series of Wolvercote bathing area E. coli levels and the next closest E. coli site on the same graph
# Save the datasetID and siteID for each
# wolvercote_bathing_sites = sites_discoveries[0].sites.loc[0]
# wolvercote_timeseries_info = (datasets_with_ecoli.loc[0, "dataset"],
#                               sites_discoveries[0].sites.loc[0, "properties_id"],
#                               datasets_with_ecoli.loc[0, "id"]) # datasetID, siteID, determinand
# print(wolvercote_timeseries_info)

      # "code": "ea_bathing_water",
      #               "id": "EC",
      #               "column": "escherichia coli count"

row_nearest = within_10km_wolvercote.head(1).index
nearest_siteID = sites_discoveries[1].sites.loc[row_nearest[0], "properties_id"]
print(nearest_siteID)
nearest_timeseries_info = (datasets_with_ecoli.loc[1, "dataset"],
                              nearest_siteID,
                              datasets_with_ecoli.loc[1, "id"]) # datasetID, siteID, determinand
print(nearest_timeseries_info)

# get the timeseries for each
# wolvercote_ecoli_timeseries = loader.load_timeseries_determinand(wolvercote_timeseries_info[0], wolvercote_timeseries_info[1], wolvercote_timeseries_info[2])
# nearest_ecoli_timeseries = loader.load_timeseries_determinand(nearest_timeseries_info[0], nearest_timeseries_info[1], nearest_timeseries_info[2])

loader.load_timeseries_determinand('wtrt', '1743499042454x452904883018006500', "EC")


1743499042454x452904883018006500
('wtrt', '1743499042454x452904883018006500', 'EC')
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/timeseries
Data storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data
Endpoint storage directory created at /Users/jeannielynnmoulton/Github/oxford-rivers-portal-api-py/data/timeseries


KeyError: 'datetime'